In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import sequence

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

17464789/17464789 [==============================] - 0s 0us/step


In [4]:
word_index = tf.keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


In [5]:
word_index = {k:(v+3) for k, v in word_index.items()}

word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3


reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [6]:
# 這邊可以調
review_length = 500

x_train = sequence.pad_sequences(x_train, maxlen = review_length)
x_test = sequence.pad_sequences(x_test, maxlen = review_length)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(25000, 500)
(25000,)
(25000, 500)
(25000,)


In [7]:
from tensorflow.keras import layers, models


In [8]:
gru = models.Sequential(
    [
        layers.Embedding(
            input_dim = vocab_size,
            output_dim = 32,
            input_length = review_length,
        ),

        layers.GRU(
            units = 32,
        ),

        layers.Dense(
            units = 1,
        ),
    ],
    name = "GRU"
)

gru.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.binary_crossentropy,
    metrics = ["accuracy"],
)

gru.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 gru (GRU)                   (None, 32)                6336      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 326369 (1.24 MB)
Trainable params: 326369 (1.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
history = gru.fit(
    x_train,
    y_train,
    batch_size = 250,
    epochs = 10,
    validation_split = 0.2,
)

Epoch 1/10
80/80 [==============================] - 37s 434ms/step - loss: 0.8868 - accuracy: 0.5544 - val_loss: 0.6365 - val_accuracy: 0.6848
Epoch 2/10
80/80 [==============================] - 34s 429ms/step - loss: 0.5467 - accuracy: 0.7539 - val_loss: 0.5877 - val_accuracy: 0.7258
Epoch 3/10
80/80 [==============================] - 36s 454ms/step - loss: 0.5556 - accuracy: 0.7555 - val_loss: 0.8999 - val_accuracy: 0.5468
Epoch 4/10
80/80 [==============================] - 36s 451ms/step - loss: 0.4732 - accuracy: 0.7835 - val_loss: 0.5445 - val_accuracy: 0.7520
Epoch 5/10
80/80 [==============================] - 36s 455ms/step - loss: 0.3794 - accuracy: 0.8498 - val_loss: 0.6267 - val_accuracy: 0.7758
Epoch 6/10
80/80 [==============================] - 35s 434ms/step - loss: 0.3421 - accuracy: 0.8755 - val_loss: 1.0565 - val_accuracy: 0.8014
Epoch 7/10
80/80 [==============================] - 37s 464ms/step - loss: 0.3404 - accuracy: 0.8917 - val_loss: 0.7197 - val_accuracy: 0.7536

In [10]:
import json

history_dict = history.history

In [11]:
json.dump(history_dict, open("/content/drive/MyDrive/Colab Notebooks/imdb/history/gru.json", 'w'))

In [12]:
gru.save("/content/drive/MyDrive/Colab Notebooks/imdb/model/gru.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
